# Dreambooth
Notebook implementation by David Bielejeski
Latest information on: https://github.com/JoePenna/Dreambooth-Stable-Diffusion

# Download with GDown

In [ ]:
# Download model with gdown

# https://github.com/JoePenna/Dreambooth-Stable-Diffusion/
!pip install gdown
# !gdown --id {insert-id-here}

# Download from HuggingFace

In [ ]:
# Download model from HuggingFace

!pip install huggingface_hub

from huggingface_hub import hf_hub_download
import joblib

REPO_ID = "id"
FILENAME = "id"

hf_hub_download(repo_id=REPO_ID, filename=FILENAME, local_dir="/workspace")

## Build Environment

In [ ]:
from IPython.display import clear_output
from IPython.utils import io

print("Installing widgets library...")
with io.capture_output() as captured:
    !pip install ipywidgets
clear_output()

from JupyterNotebookHelpers.installer_progress_bar_widget import InstallerProgressBar

show_detailed_output = False
install_commands = [
    "pip install numpy==1.23.1",
    "pip install pytorch-lightning==1.7.6",
    "pip install csv-logger",
    "pip install torchmetrics==0.11.1",
    "pip install torch-fidelity==0.3.0",
    "pip install albumentations==1.1.0",
    "pip install opencv-python==4.7.0.72",
    "pip install pudb==2019.2",
    "pip install omegaconf==2.1.1",
    "pip install pillow==9.4.0",
    "pip install einops==0.4.1",
    "pip install transformers==4.25.1",
    "pip install kornia==0.6.7",
    "pip install diffusers[training]==0.3.0",
    "pip install captionizer==1.0.1",
    "pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers",
    "pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip",
    "pip install -e .",
    "pip install huggingface_hub",
    "pip install gitpython",
]

progress_bar = InstallerProgressBar()
progress_bar.show(install_commands)

try:
    for i, install_command in enumerate(install_commands):
        if show_detailed_output:
            !{install_command}
        else:
            with io.capture_output() as captured:
                !{install_command}
            print(f"☑ '{install_command.replace('pip install ', '')}' installation complete.")

        # update the progress bar
        progress_bar.increment(i)

    progress_bar.close()
    print("✅ All Dependencies successfully installed.  Proceed to model download.")

except:
    progress_bar.close()
    print("❌ Error installing dependencies")

# Download a 1.* model

In [ ]:
from JupyterNotebookHelpers.download_model import DownloadModel

download_model = DownloadModel()
download_model.show_form()

# Download Regularization imagesThen Edit the 2 files in the 'joepenna-dreambooth-configs' folder
Change the 'regularization_image_folder_path' to '512x512/{tag}_ddim

In [ ]:
reg_upload_folder = '512x512'
tag = 'anything'
reg_folder = '512x512Reg'
!mkdir -p {reg_folder}
reg_zip_filename = '512x512.zip'
!mv {reg_zip_filename} {reg_folder}
import zipfile as zf
files = zf.ZipFile(f'{reg_folder}/' + f'{reg_zip_filename}','r')
files.extractall('/workspace/Dreambooth-Stable-Diffusion/512x512Reg')
files.close()
!mv /workspace/Dreambooth-Stable-Diffusion/512x512Reg/512x512 /workspace/Dreambooth-Stable-Diffusion/512x512Reg/{tag}_ddim


## Dreambooth Training Environment Setup

In [ ]:
from JupyterNotebookHelpers.setup_training import SetupTraining

setup_training = SetupTraining()

# Pass our selected model to the training form
model_to_train_on = download_model.get_selected_model()

# Show the form.
setup_training.show_form(
    selected_model=model_to_train_on
)

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
from IPython.display import clear_output
import os
import shutil

clear_output()

# Load the config generated on the previous step
active_config_path = "./joepenna-dreambooth-configs/active-config.json"

if os.path.exists(active_config_path):
    # Cleanup garbage files if they exist
    leftover_training_files = ["./training_images/.ipynb_checkpoints", "./regularization_images/.ipynb_checkpoints"]
    for i, ipynb_checkpoint_folder in enumerate(leftover_training_files):
        if os.path.exists(ipynb_checkpoint_folder):
            shutil.rmtree(ipynb_checkpoint_folder)

    # Train
    !python "main.py" --config_file_path "{active_config_path}"

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Optional - Upload to HuggingFace
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
Create a hugging face repo
Add read and write access tokens for your huggingface account, for logging in with the next step
* `python -m pip install huggingface_hub`
* `huggingface-cli login`
Create a git sshkey for your account
* `ssh-keygen -t ed25519 -C "my_email@gmail.com"`
* `cat /root/.ssh/id_ed25519.pub`
Add ssh public key to hugging face account
Clone your hugging face repo
* `git clone git@hf.co:<your-username>/<your-model-name>`
* `cd <your-model-name>`
Install git-lfs
* `apt-update`
* `apt install git-lfs`
Init git-lfs
* `git lfs install`
Note that if your files are larger than 5GB you’ll also need to run:
* `huggingface-cli lfs-enable-largefiles .`
Push to huggingface
* `git add .`
* `git commit -m "First model version"`
* `git push`


# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`